In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore") 

In [25]:
df = pd.read_csv("../data/raw/exploratory.csv")
df.head(5)

,id,sid,sex,cycle,childs,childbirth,dob,deadwood_days,pregancy_days,group_id,group_past,event,milking_days,date,remark
0,1064,1064,F,1,FA,1,1/8/2020,0,0,9,9,ОСЕМЕН,236,7/1/2022,1H15132
1,1064,1064,F,1,FA,1,1/8/2020,0,0,9,9,НА_СХЕМУ,236,7/1/2022,9
2,1185,1185,F,1,FA,1,1/20/2020,0,49,9,43,ОСЕМЕН,213,7/1/2022,1H15184
3,1185,1185,F,1,FA,1,1/20/2020,0,49,9,43,НА_СХЕМУ,213,7/1/2022,9
4,1260,1260,F,1,FA,1,1/27/2020,0,49,5,5,НА_СХЕМУ,205,7/1/2022,9


# Предобработка

In [26]:
df=df.drop(columns=["sid"])

## Проверка на пустые значения

In [27]:
for column in df:
    print(column, ":", len(df.loc[(df[column].isnull()) | (df[column]=='-')].index))

id : 0
sex : 0
cycle : 0
childs : 5310
childbirth : 0
dob : 0
deadwood_days : 0
pregancy_days : 0
group_id : 0
group_past : 0
event : 0
milking_days : 0
date : 0
remark : 644


В childs ставится NaN если ещё не было ребенка или животное мужского пола.
В remark стоит - если примечание не требуется

## Приведение к числовым значениям

In [28]:
df.dtypes

id                int64
sex              object
cycle             int64
childs           object
childbirth        int64
dob              object
deadwood_days     int64
pregancy_days     int64
group_id          int64
group_past        int64
event            object
milking_days      int64
date             object
remark           object
dtype: object

### Пол и дети

In [29]:
df['sex'] = df['sex'].apply(lambda x: 1 if x =='F' else 0)

In [30]:
df['childs_fa'] = df['childs'].apply(lambda x: str(x).count('FA'))
df['childs_ma'] = df['childs'].apply(lambda x: str(x).count('MA'))
df['childs_fd'] = df['childs'].apply(lambda x: str(x).count('FD'))
df['childs_md'] = df['childs'].apply(lambda x: str(x).count('MD'))

In [31]:
#df=df.drop(columns=["childs"])
df.dtypes

id                int64
sex               int64
cycle             int64
childs           object
childbirth        int64
dob              object
deadwood_days     int64
pregancy_days     int64
group_id          int64
group_past        int64
event            object
milking_days      int64
date             object
remark           object
childs_fa         int64
childs_ma         int64
childs_fd         int64
childs_md         int64
dtype: object

### Дата рождения

In [32]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df['dob']  = pd.to_datetime(df['dob'], infer_datetime_format=True)
df['age']  = ((df['date'] - df['dob']) / np.timedelta64(1, 'D')).astype(int)

In [33]:
df=df.drop(columns=["dob"])
df.dtypes

id                        int64
sex                       int64
cycle                     int64
childs                   object
childbirth                int64
deadwood_days             int64
pregancy_days             int64
group_id                  int64
group_past                int64
event                    object
milking_days              int64
date             datetime64[ns]
remark                   object
childs_fa                 int64
childs_ma                 int64
childs_fd                 int64
childs_md                 int64
age                       int64
dtype: object

### Дата события

In [34]:
month=['jan','feb','mar','apr','may','jun', 'jul', 'aug', 'sep', 'oct', 'nov',' dec']
for m in range (12):
    df[month[m]]=df['date'].apply(lambda x: 1 if x.month == m + 1 else 0)
df.head()

,id,sex,cycle,childs,childbirth,deadwood_days,pregancy_days,group_id,group_past,event,...,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,1064,1,1,FA,1,0,0,9,9,ОСЕМЕН,...,0,0,0,0,1,0,0,0,0,0
1,1064,1,1,FA,1,0,0,9,9,НА_СХЕМУ,...,0,0,0,0,1,0,0,0,0,0
2,1185,1,1,FA,1,0,49,9,43,ОСЕМЕН,...,0,0,0,0,1,0,0,0,0,0
3,1185,1,1,FA,1,0,49,9,43,НА_СХЕМУ,...,0,0,0,0,1,0,0,0,0,0
4,1260,1,1,FA,1,0,49,5,5,НА_СХЕМУ,...,0,0,0,0,1,0,0,0,0,0


## Обработка событий

In [35]:
from enum import Enum

class EventType(Enum):
    
    OTHER   = -1
    #-------------
    ABORT   = 1  # 
    TOSCM   = 2  #
    NULSCM  = 3  #
    DNB     = 4  #
    BRED    = 5  #
    FRESH   = 6  #
    PREG    = 7  #
    PREGBEF = 8  #
    DRY     = 9  #
    DRY2    = 10 #
    OPEN    = 11 #
    #-------------
    WEIGHT  = 12 #
    DEAD    = 13 #
    MOVE    = 14 #
    SOLD    = 15 #
    #--------------
    VAC     = 16 #
    VACVIR  = 17 #
    FOOTRIM = 18 #
    HEALTH  = 19 #
    BROKE   = 20 #
    POT     = 21 #
    ILLMISC = 22 #
    LAME    = 23 #
    KETOS   = 24 #
    MAST    = 25 #
    METR    = 26 #
    PARES   = 27 #
    RP      = 28 #
    
    def __str__(self):
        return self.name
    
    @staticmethod
    def from_str(label):
        EventNames = {
         EventType.ABORT   : ["ABORT", "АБОРТ"],  
         EventType.TOSCM   : ["TOSCM", "НА_СХЕМУ"], 
         EventType.NULSCM  : ["NULSCM","СО_СХЕМЫ"],
         EventType.DNB     : ["DNB", "НЕОСЕМ"],
         EventType.BRED    : ["BRED", "ОСЕМЕН"], 
         EventType.FRESH   : ["FRESH", "ОТЕЛ"],  
         EventType.PREG    : ["PREG", "СТЕЛН"], 
         EventType.PREGBEF : ["PREGBEF", "СТЕЛНДО"],
         EventType.DRY     : ["DRY", "СУХОСТ"],
         EventType.DRY2    : ["DRY2", "СУХ2"],
         EventType.OPEN    : ["OPEN", "ЯЛОВАЯ"],
         #-------
         EventType.WEIGHT  : ["WEIGHT", "ВЕС"],
         EventType.DEAD    : ["DEAD", "ПАЛА"],
         EventType.MOVE    : ["MOVE", "ПЕРЕВОД"],
         EventType.SOLD    : ["SOLD", "ПРОДАНА"],
         #-------
         EventType.VAC     : ["VAC", "ВАКЦИН"],
         EventType.VACVIR  : ["VACVIR", "ВАКВИРУС"],
         EventType.FOOTRIM : ["FOOTRIM", "РАСЧКОП"],
         EventType.HEALTH  : ["HEALTH", "WELL", "ЗДОРОВА"],
         EventType.BROKE   : ["BROKE", "ДЕФЕКТ"],
         EventType.POT     : ["POT", "ПРОФОТ"],
         EventType.ILLMISC : ["ILLMISC", "БОЛЕЗНЬ"],
         EventType.LAME    : ["LAME", "ХРОМОТА"],
         EventType.KETOS   : ["KETOS", "КЕТОЗ"],
         EventType.MAST    : ["MAST", "МАСТИТ"],
         EventType.METR    : ["METR", "МЕТРИТ"],
         EventType.PARES   : ["PARES", "ПАРЕЗ"],
         EventType.RP      : ["RP", "ПОСЛЕД"],
        }
        for ev in EventNames.keys():
            if label in EventNames[ev]:
                return ev
        return Event(-1)

In [36]:
df['event'] = df['event'].apply(lambda x: EventType.from_str(x))
df.head()

,id,sex,cycle,childs,childbirth,deadwood_days,pregancy_days,group_id,group_past,event,...,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,1064,1,1,FA,1,0,0,9,9,BRED,...,0,0,0,0,1,0,0,0,0,0
1,1064,1,1,FA,1,0,0,9,9,TOSCM,...,0,0,0,0,1,0,0,0,0,0
2,1185,1,1,FA,1,0,49,9,43,BRED,...,0,0,0,0,1,0,0,0,0,0
3,1185,1,1,FA,1,0,49,9,43,TOSCM,...,0,0,0,0,1,0,0,0,0,0
4,1260,1,1,FA,1,0,49,5,5,TOSCM,...,0,0,0,0,1,0,0,0,0,0


### Примечания к событиям
События делим на три группы:
1. События без примечаний
2. События, примечания к которым обрабатываются однообразно (например примечания --- номер протокола или id семени)
3. События, примечания к которым обрабытваются сложным образом.

Чтобы выбрать событие третьего типа, надо выбрать правило которым его надо обработать, а затем собственно произвести обработку.

In [37]:
for event in EventType:
    print(event, df[df['event'] == event]['remark'].unique().tolist())
    print('---------------------------------------------------------------------------')

OTHER []
---------------------------------------------------------------------------
ABORT ['94 ДНИ', '95 ДНИ', '81 ДНИ', 'БЕЗ ЖТ', '213 ДНИ', '172 ДНИ', 'ЖТ', 'ЛК', '241 ДНИ', '160 ДНИ', '83 ДНИ', '130 ДНИ', '84 ДНИ', '91 ДНИ', '7/25/2022', '214 ДНИ', 'БРАК', '54 ДНИ']
---------------------------------------------------------------------------
TOSCM ['9', '13', '40', '31', '29', '30', '38', '33', '46', '34', '36', '55', '60', '53', '37', '52', '42', '43', '58', '47', '48', '11', '2', '5', '15', '41', '49']
---------------------------------------------------------------------------
NULSCM ['-']
---------------------------------------------------------------------------
DNB ['ВЫМЯ', '-', 'МЕЛКАЯ', 'ВОСПР', 'НОГИ', 'ГИНЕКОЛ', 'НОГИПУТЫ', 'НАДОЙ', 'АТП', 'АБЦЕС', 'БРАК', 'СУСТАВ', 'ЗООТЕХ', 'ABC', 'МАСТИТ', 'ПНЕВМО', 'ЗДОРОВЬЕ']
---------------------------------------------------------------------------
BRED ['1H15132', '1H15184', '501H14018', '1H15689', '1H15461', '501H15515', '7H14578',

Некоторые замечания:
1. В примечаниях к событиям категории OPEN встречается 'БПАК' что судя по всему является опечаткой от 'БРАК'
2. В примечаниях к событиям категории OPEN встречается 'БЖТ' и 'БЕЗ ЖТ' и вероятно это одно и тоже
3. В примечаниях к событиям категории SOLD причина продажи и код причины продажи не находятся во взаимнооднозначном соответствии

In [38]:
import re
countday=re.compile('[0-9]* ДНИ')

df['remark'] = df.apply(lambda row: 'БРАК' if (row.event == EventType.OPEN and row.remark == 'БПАК') else row.remark, axis=1)
df['remark'] = df.apply(lambda row: 'БЖТ' if  (row.event == EventType.OPEN and row.remark == 'БЕЗ ЖТ') else row.remark, axis=1)
df['remark'] = df.apply(lambda row: row.remark.split(';')[1].strip().rstrip() if row.event == EventType.SOLD else row.remark, axis=1)
df['remark'] = df.apply(lambda row: row.remark.split(';')[1].strip().rstrip() if row.event == EventType.DEAD else row.remark, axis=1)

EVENT_EMPTY  = [EventType.OTHER, EventType.NULSCM, EventType.FRESH, EventType.DRY2]
EVENT_UNIQUE = [EventType.TOSCM, EventType.DNB, EventType.BRED, EventType.SOLD, EventType.DEAD, EventType.DRY, EventType.OPEN, EventType.VAC, EventType.VACVIR, EventType.FOOTRIM, EventType.POT, EventType.ILLMISC, EventType.METR, EventType.PARES, EventType.RP]

for event in EVENT_EMPTY:
    df[str(event)] = df.apply(lambda row: 1 if row.event == event else 0, axis=1)

for event in EVENT_UNIQUE:
    protocols = df[df['event'] == event]['remark'].unique().tolist()
    i = 0
    for p in protocols:
        p_feature_name = str(event) + '_' + str(i); i += 1;
        df[p_feature_name] = df.apply(lambda row: 1 if row.event == event and row.remark == p else 0, axis=1)

    
#parse ABORT:
df['ABORT'] = df.apply(lambda row: 1 if row.event == EventType.ABORT else 0, axis=1)
df['ABORT_DAYS'] = df.apply(lambda row: int(row.remark[0:-4]) if (row.event == EventType.ABORT and countday.match(row.remark)) else 0, axis=1)
df['ABORT_YB']   = df.apply(lambda row: 1 if (row.event == EventType.ABORT and row.remark == 'ЖТ') else 0, axis=1)
df['ABORT_LK']   = df.apply(lambda row: 1 if (row.event == EventType.ABORT and row.remark == 'ЛК') else 0, axis=1)
df['ABORT_NYB']  = df.apply(lambda row: 1 if (row.event == EventType.ABORT and row.remark == 'БЕЗ ЖТ') else 0, axis=1)
df['ABORT_BRK']  = df.apply(lambda row: 1 if (row.event == EventType.ABORT and row.remark == 'БРАК') else 0, axis=1)
    
#parse PREG:
df['PREG_EVENT_DAYS'] = df.apply(lambda row: int(row.remark[0:-4]) if (row.event == EventType.PREG and countday.match(row.remark)) else 0, axis=1)

#parse PREGBEF:
df['PREGBEF_EVENT_DAYS'] = df.apply(lambda row: int(row.remark[0:-4]) if (row.event == EventType.PREGBEF and countday.match(row.remark)) else 0, axis=1)

#parse WEIGHT:
#df['WEIGHT']  = df.apply(lambda row: int(row.remark) if (row.event == EventType.WEIGHT) else 0, axis=1)
df.head()

,id,sex,cycle,childs,childbirth,deadwood_days,pregancy_days,group_id,group_past,event,...,RP_0,RP_1,ABORT,ABORT_DAYS,ABORT_YB,ABORT_LK,ABORT_NYB,ABORT_BRK,PREG_EVENT_DAYS,PREGBEF_EVENT_DAYS
0,1064,1,1,FA,1,0,0,9,9,BRED,...,0,0,0,0,0,0,0,0,0,0
1,1064,1,1,FA,1,0,0,9,9,TOSCM,...,0,0,0,0,0,0,0,0,0,0
2,1185,1,1,FA,1,0,49,9,43,BRED,...,0,0,0,0,0,0,0,0,0,0
3,1185,1,1,FA,1,0,49,9,43,TOSCM,...,0,0,0,0,0,0,0,0,0,0
4,1260,1,1,FA,1,0,49,5,5,TOSCM,...,0,0,0,0,0,0,0,0,0,0


### Целевые события
Теперь разберемся с целевыми событиями: мастит и хромота

In [39]:
def prot_id(s):
    return s[0:s.find('_'):]

def prot_num(s):
    m = []
    t = s[s.find('_') + 1::]
    for i in range(len(t)):
        if t[i] == '_':
            a = int(t[i - 1])
            b = int(t[i + 1])
            for j in range(a+1, b):
                m.append(j)
        elif t[i].isdigit():
            m.append(int(t[i]))
    return m

# Протоколы лечения хромоты
print(set(map(prot_id, df[df['event'] == EventType.LAME]['remark'].unique().tolist())))
# Протоколы лечения мастита
print(set(map(prot_id, df[df['event'] == EventType.MAST]['remark'].unique().tolist())))

{'V2', 'N3', 'А', 'H1', 'H2', 'N1', 'HP1', 'V3', 'HP2', 'N2', 'V1', 'M1'}
{'КМ2', 'ТМ3', 'КМ5', 'СКМ6', 'ТМ9', 'КМ4', 'КМ1', 'ТМ5', 'ТМ2', 'М', 'КМ6', 'ТМ1', 'КМ3'}


In [40]:
def parse_LAME():
    protocols = set(map(prot_id, df[df['event'] == EventType.LAME]['remark'].unique().tolist()))
    for p in protocols:
        for i in range(1, 5):
            feature_name = 'LAME_'+ p + '_' + str(i)
            df[feature_name] =  df.apply(lambda row: 1 if ((row.event == EventType.LAME) and (p == prot_id(row.remark)) and (i in prot_num(row.remark))) else 0, axis=1)

def parse_MAST():
    protocols = set(map(prot_id, df[df['event'] == EventType.MAST]['remark'].unique().tolist()))
    for p in protocols:
        for i in range(1, 5):
            feature_name = 'MAST_'+ p + '_' + str(i)
            df[feature_name] =  df.apply(lambda row: 1 if ((row.event == EventType.MAST) and (p == prot_id(row.remark)) and (i in prot_num(row.remark))) else 0, axis=1)
parse_LAME()
parse_MAST()

In [42]:
df.head()

,id,sex,cycle,childs,childbirth,deadwood_days,pregancy_days,group_id,group_past,event,...,MAST_КМ6_3,MAST_КМ6_4,MAST_ТМ1_1,MAST_ТМ1_2,MAST_ТМ1_3,MAST_ТМ1_4,MAST_КМ3_1,MAST_КМ3_2,MAST_КМ3_3,MAST_КМ3_4
0,1064,1,1,FA,1,0,0,9,9,BRED,...,0,0,0,0,0,0,0,0,0,0
1,1064,1,1,FA,1,0,0,9,9,TOSCM,...,0,0,0,0,0,0,0,0,0,0
2,1185,1,1,FA,1,0,49,9,43,BRED,...,0,0,0,0,0,0,0,0,0,0
3,1185,1,1,FA,1,0,49,9,43,TOSCM,...,0,0,0,0,0,0,0,0,0,0
4,1260,1,1,FA,1,0,49,5,5,TOSCM,...,0,0,0,0,0,0,0,0,0,0
